In [11]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
import tensorflow as tf

from scipy.special import softmax

import nltk
nltk.download('stopwords')
nltk.download('tokenizers')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
data = pd.read_csv('/content/RedditNews.csv')

In [14]:
print(data.shape)
print(data.info())


(73608, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73608 entries, 0 to 73607
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    73608 non-null  object
 1   News    73608 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB
None


In [15]:
data.head(2)

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host


## Pré-processamento

### As células abaixo estão comentadas porque não são necessárias quando da utilização do modelo pré-treinado.
O pré-processamento deverá ser realizado se o modelo for elaborado desde o início.

In [16]:
# def basic_cleaning(sentence):
#     sentence = sentence.lower()
#     sentence = ''.join(char for char in sentence if not char.isdigit())

#     for punctuation in string.punctuation:
#         sentence = sentence.replace(punctuation, ' ')

#     sentence = sentence.strip()

#     return sentence


In [17]:
# data['News_cleaned'] = data['News'].apply(basic_cleaning)
# data.head(2)

In [18]:
# data['tokens'] = data['News_cleaned'].apply(word_tokenize)


In [19]:
# stop_words = set(stopwords.words('english'))

# def remove_stopwords(tokens):
#     return [w for w in tokens if w not in stop_words]

# data['tokens_cleaned'] = data['tokens'].apply(remove_stopwords)

In [20]:
# def lemmatize_tokens(tokens, pos='n'):
#     lemmatizer = WordNetLemmatizer()
#     if pos == 'v':
#         return [lemmatizer.lemmatize(word, pos='v') for word in tokens]
#     else:
#         return [lemmatizer.lemmatize(word, pos='n') for word in tokens]

In [21]:
# data['tokens_lemmatized'] = data.tokens_cleaned.apply(lemmatize_tokens)

In [22]:
# # Apenas para conferência se as alterações foram efetuadas com sucesso. A coluna tokens_cleaned deve ser diferente da tokens_lemmatized, pelos menos em grande parte.
# print((data['tokens_cleaned'] == data['tokens_lemmatized']).unique())

## Modelo - Roberta Finetued Financial News Sentiment Analysis

In [23]:
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [24]:

sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)
sentiment_task('tokens')


[{'label': 'neutral', 'score': 0.9998683929443359}]

## Aplicação do modelo ao dataset


In [25]:
def analyze(text):
    result = sentiment_task(text)
    return result[0]

In [26]:
%time
data_sentiment = data['News'].apply(analyze)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [27]:
data_sentiment.head()

0    {'label': 'neutral', 'score': 0.9997691512107849}
1    {'label': 'neutral', 'score': 0.9996070265769958}
2    {'label': 'neutral', 'score': 0.9990409016609192}
3    {'label': 'neutral', 'score': 0.9998262524604797}
4    {'label': 'neutral', 'score': 0.9993951320648193}
Name: News, dtype: object

In [28]:
data_sentiment_label = data_sentiment.apply(lambda x: x['label'])
data_sentiment_score = data_sentiment.apply(lambda x: round(x.get('score', 0), 2) if isinstance(x, dict) else None)

In [29]:
data_sentiment_label

0         neutral
1         neutral
2         neutral
3         neutral
4         neutral
           ...   
73603     neutral
73604     neutral
73605    negative
73606     neutral
73607     neutral
Name: News, Length: 73608, dtype: object

In [32]:
zipped = list(zip(data_sentiment_label, data_sentiment_score))

In [33]:
zipped_df = pd.DataFrame(zipped, columns=['Label', 'Score'])


,Label,Score
0,neutral,1.00
1,neutral,1.00
2,neutral,1.00
3,neutral,1.00
4,neutral,1.00
...,...,...
73603,neutral,0.91
73604,neutral,0.99
73605,negative,0.98
73606,neutral,1.00


In [34]:
data_df = pd.concat([data, zipped_df], axis=1, join='outer', ignore_index=False)
data_df.shape


(73608, 4)

In [37]:
data_df.to_csv('reddit_data',index=False)